# Optimal Resource Allocation in Public and Private Cloud

## Define System Model (MDP)

In [1]:
class System: # represents the MDP
    
    def __init__(self, E, Pefc) -> None:
        self.E = E # total VMs at edge
        self.H = [] # Record List
        self.Pe, self.Pf, self.Pc = Pefc # cost parameters
        self.verbose=False
        
    def reset(self):
        self.et = self.E # currently avaialble VMs
        self.H.clear() # clear record list
        self.t = 0   # time step
        return self

    def step(self, d, l, a): 
        # represents handling one user demand
        self.t+=1
        verb = self.verbose
        # d = no of vms requesterd, 
        # l = duration requested
        # a = action (ratio of vms allocated from cloud)
        if verb: 
            print(f'{self.t=}')
            print(f'\tDemand, {d, l}')
            print(f'\tAction, {a}')
        
        c = int(d*a) # vms allocated from cloud
        e = d - c    # vms allocated from edge
        r = self.et - e # remaining VMs after allocation
        # check if enough vms available?
        if r<0:
            e = self.et # allocated all from edge
            c += (-r) # take remaining from cloud
        self.et -= e
        if verb: 
            print(f'\tAllocated from Cloud, {c=}')
            print(f'\tAllocated from Edge, {e=}')
            print(f'\tRemaining, {self.et=}')
        # generate allocation record 
        if e > 0:
            self.H.append([e, l]) #<-- appending a list to a list
            if verb:
                print(f'\t\tAllocation Record, {[e,l]}')
                print(f'\t\tAllocation Record List {self.H=}')    

        # cost at edge node
        Ce = (self.E-self.et)*self.Pf + (self.et)*self.Pe

        # cost at private cloud
        Cpri = c*self.Pc + Ce
        if verb:
            print(f'\tCost At Edge Node, {Ce=}')
            print(f'\tCost At Private Cloud, {Cpri=}')

        #<------------------------------- round 

        # update allocation record
        for el in self.H: el[-1]-=1
        # release exsisting VMs
        # remove completed records
        i=0
        n=0 # no of busy vms (waiting to be released)
        while i < len(self.H):
            if self.H[i][-1]==0: 
                n+=self.H[i][0] # reclaim
                del self.H[i]   # remove
            else: i+=1 # skip
        
        if verb:
            print(f'\tUpdated Allocation Record List {self.H=}')  
            print(f'\tVMs waiting to be released {n=}')

        self.et += n # update available
        if verb: print(f'\tVMs available at next time slot {self.et=}')
        return Cpri


## Define Environment for RL

In [2]:
import numpy as np
class Environment: # Encapsulates an MDP for agent interaction
    def __init__(self, E, D, L, T, Pefc, seed=None ) -> None:
        self.E, self.D, self.L, self.T = E, D, L, T

        self.nS = self.T*self.D*self.L*(self.E+1)
        self.nD = self.D*self.L
        self.DL = np.array([ [(d,l) for l in range(1, L+1)] for d in range(1, D+1)])
        self.DL_ = self.DL.reshape(self.DL.shape[0]*self.DL.shape[1], self.DL.shape[2])
        self.dls = np.arange(len(self.DL_))
        self.S = np.zeros(4, dtype=int) #(etdl)
        self.A=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        self.nA = len(self.A)
        self.sim = System(E, Pefc)
        self.rng = np.random.default_rng(seed)
    
    def reset(self):
        self.sim.reset()
        self.S[0] = self.sim.et
        self.S[1] = self.sim.t
        self.S[2:] = self.DL_[self.rng.choice(self.dls)]
        return self.S

    def step(self, action):
        #print(f'{self.S=}, {self.A=}::{action=}')
        cost = self.sim.step(d=self.S[2], l=self.S[3], a=self.A[action])
        done = not(self.sim.t < self.T)
        self.S[0] = self.sim.et
        self.S[1] = self.sim.t
        self.S[2:] = 0 if done else self.DL_[self.rng.choice(self.dls)]
        return self.S, float(-cost), done

## Initialize Environment

In [3]:
env = Environment(E=19, D=4, L=3, T=15, Pefc=(0.03, 0.20, 3.00), seed=13)
env.sim.verbose=False
env.__dict__

{'E': 19,
 'D': 4,
 'L': 3,
 'T': 15,
 'nS': 3600,
 'nD': 12,
 'DL': array([[[1, 1],
         [1, 2],
         [1, 3]],
 
        [[2, 1],
         [2, 2],
         [2, 3]],
 
        [[3, 1],
         [3, 2],
         [3, 3]],
 
        [[4, 1],
         [4, 2],
         [4, 3]]]),
 'DL_': array([[1, 1],
        [1, 2],
        [1, 3],
        [2, 1],
        [2, 2],
        [2, 3],
        [3, 1],
        [3, 2],
        [3, 3],
        [4, 1],
        [4, 2],
        [4, 3]]),
 'dls': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 'S': array([0, 0, 0, 0]),
 'A': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
 'nA': 11,
 'sim': <__main__.System at 0x1b26a1f1490>,
 'rng': Generator(PCG64) at 0x1B26A241B60}

## Q-Learning Algorithm

In [4]:
def Q_Learning(
        mdp,    # formulated mdp 
        πe,     # behaviour policy
        α,      # learning rate
        γ,      # discount factor
        N,      # number of learning rounds
    ):
    Q = {}  # initialize Q-Table
    s = tuple(mdp.reset()) # reset the mdp and obtain initial state
    if s not in Q: 
        # add obtained state to Q-Table and initalize values as zeros
        Q[s] = [0.0 for _ in range(mdp.nA)] 

    for n in range(N): # learning loop
        a = πe(s, Q) # select action using behaviour policy
        s_, r,  done = mdp.step(a) # obtain reward(r) and next state(s_)
        s_ = tuple(s_)
        if s_ not in Q: 
            # add obtained next state to Q-Table and initalize values as zeros
            Q[s_] = [0.0 for _ in range(mdp.nA)]
        
        Q[s][a] = (1-α) * Q[s][a] + (α) * (r + γ * max(Q[s_])) # update Q-values
        if done: # final state reached?
            s = tuple(mdp.reset()) # reset the mdp
            if s not in Q: Q[s] = [0.0 for _ in range(mdp.nA)]
        else:
            s = s_  # continue to next time step
    return Q


## Epsilon Greedy Exploration

In [5]:
class EpsilonGreedyPolicy:
    def __init__(self, n_actions, seed=None) -> None: 
        self.n_actions, self.eps, self.decay, self.rng = n_actions, 1.0, 0.0, np.random.default_rng(seed)
    def set_epsilon(self, eps): self.eps = eps
    def set_decay(self, delta): self.delta = delta
    def __call__(self, state, Q): 
        a = self.rng.integers(0,self.n_actions) if self.rng.random()<self.eps else np.argmax(Q[state])
        self.eps-=self.delta
        return a


### Run Q-Learning Algorithm

In [6]:
behave = EpsilonGreedyPolicy(env.nA, seed=2654)
behave.set_epsilon(1.0)
behave.set_decay(1/100000)
q = Q_Learning(env, behave, 0.5, 1.0, 100000)

### Learnt Q-Table

In [7]:
# q

### Learnt policy

In [27]:
def Q_Policy(mdp, Q):
    s = tuple(mdp.reset())
    done = False
    cost = 0.0
    actions = []
    while not done:
        q = Q[s]
        a = np.argmax(q)
        s_, r,  done = mdp.step(a)
        s_ = tuple(s_)
        cost+= (-r) # -(cost_at_private_cloud:c_pri)
        actions.append(env.A[a])
        print(f'{s=}, {a=}:{env.A[a]}, {r=:.5f}, {s_=}, {done=}, {cost=:.5f}')
        s= s_
    return cost, actions

### Run Learnt policy

In [30]:
ret, acts = Q_Policy(env, q)
print(f'actions: {acts}')
print(f'cost {ret:.5f}')

s=(19, 0, 2, 2), a=3:0.3, r=-0.91000, s_=(17, 1, 4, 1), done=False, cost=0.91000
s=(17, 1, 4, 1), a=0:0.0, r=-1.59000, s_=(19, 2, 2, 1), done=False, cost=2.50000
s=(19, 2, 2, 1), a=5:0.5, r=-3.74000, s_=(19, 3, 1, 2), done=False, cost=6.24000
s=(19, 3, 1, 2), a=7:0.7, r=-0.74000, s_=(18, 4, 1, 1), done=False, cost=6.98000
s=(18, 4, 1, 1), a=7:0.7, r=-0.91000, s_=(19, 5, 2, 3), done=False, cost=7.89000
s=(19, 5, 2, 3), a=8:0.8, r=-3.74000, s_=(18, 6, 3, 1), done=False, cost=11.63000
s=(18, 6, 3, 1), a=0:0.0, r=-1.25000, s_=(18, 7, 2, 3), done=False, cost=12.88000
s=(18, 7, 2, 3), a=3:0.3, r=-1.08000, s_=(17, 8, 2, 3), done=False, cost=13.96000
s=(17, 8, 2, 3), a=0:0.0, r=-1.25000, s_=(15, 9, 3, 3), done=False, cost=15.21000
s=(15, 9, 3, 3), a=1:0.1, r=-1.76000, s_=(14, 10, 2, 2), done=False, cost=16.97000
s=(14, 10, 2, 2), a=3:0.3, r=-1.76000, s_=(14, 11, 4, 2), done=False, cost=18.73000
s=(14, 11, 4, 2), a=2:0.2, r=-2.10000, s_=(15, 12, 4, 1), done=False, cost=20.83000
s=(15, 12, 4, 1)

# Environment Simulation

(for worked out examples)

In [10]:
S = System(E=80, Pefc=(0.03, 0.20, 3.00)).reset()
S.verbose=True
print(S.__dict__)

{'E': 80, 'H': [], 'Pe': 0.03, 'Pf': 0.2, 'Pc': 3.0, 'verbose': True, 'et': 80, 't': 0}


In [11]:
cost = S.step(d=30, l=2, a=0.4)
print(S.__dict__)

self.t=1
	Demand, (30, 2)
	Action, 0.4
	Allocated from Cloud, c=12
	Allocated from Edge, e=18
	Remaining, self.et=62
		Allocation Record, [18, 2]
		Allocation Record List self.H=[[18, 2]]
	Cost At Edge Node, Ce=5.46
	Cost At Private Cloud, Cpri=41.46
	Updated Allocation Record List self.H=[[18, 1]]
	VMs waiting to be released n=0
	VMs available at next time slot self.et=62
{'E': 80, 'H': [[18, 1]], 'Pe': 0.03, 'Pf': 0.2, 'Pc': 3.0, 'verbose': True, 'et': 62, 't': 1}


In [12]:
cost = S.step(d=10, l=1, a=0.7)
print(S.__dict__)

self.t=2
	Demand, (10, 1)
	Action, 0.7
	Allocated from Cloud, c=7
	Allocated from Edge, e=3
	Remaining, self.et=59
		Allocation Record, [3, 1]
		Allocation Record List self.H=[[18, 1], [3, 1]]
	Cost At Edge Node, Ce=5.970000000000001
	Cost At Private Cloud, Cpri=26.97
	Updated Allocation Record List self.H=[]
	VMs waiting to be released n=21
	VMs available at next time slot self.et=80
{'E': 80, 'H': [], 'Pe': 0.03, 'Pf': 0.2, 'Pc': 3.0, 'verbose': True, 'et': 80, 't': 2}


In [13]:
cost = S.step(d=20, l=2, a=0.8)
print(S.__dict__)

self.t=3
	Demand, (20, 2)
	Action, 0.8
	Allocated from Cloud, c=16
	Allocated from Edge, e=4
	Remaining, self.et=76
		Allocation Record, [4, 2]
		Allocation Record List self.H=[[4, 2]]
	Cost At Edge Node, Ce=3.08
	Cost At Private Cloud, Cpri=51.08
	Updated Allocation Record List self.H=[[4, 1]]
	VMs waiting to be released n=0
	VMs available at next time slot self.et=76
{'E': 80, 'H': [[4, 1]], 'Pe': 0.03, 'Pf': 0.2, 'Pc': 3.0, 'verbose': True, 'et': 76, 't': 3}
